In [1]:
import pandas as pd
import os  
from datetime import timedelta, datetime
from fastf1.ergast import Ergast

ergast = Ergast()

#parent_file = Path(__file__).resolve().parent.parent
parent_file = '/home/kurios/Documents/f1_analysis/'

year = 2025
race_quarter = 1
race_session = 'Q'

#year = int(input("Year ? "))
#race_quarter = int(input("Race Quarter ? (1,4) "))
#race_session = input('Session ?  (SQ, Q) ')

def race_quarter_value(race_quarter):
    match race_quarter:
        case 1 :
            return range(1, 7)
        case 2 :
            race_numbers = range(6, 13)
            return race_numbers
        case 3 :
            race_numbers = range(12, 19)
            return race_numbers
        case 4 :
            race_numbers = range(18, 25)
            return race_numbers
        
def parse_laptime_str(lap_string):
    """Parses a lap time string (MM:SS.fff) into a timedelta object."""
    if isinstance(lap_string, str) and ':' in lap_string and '.' in lap_string:
        minutes, rest = lap_string.split(':')
        seconds, milliseconds = rest.split('.')
        return timedelta(minutes=int(minutes), seconds=int(seconds), milliseconds=int(milliseconds))
    else:
        return timedelta(0)

In [2]:
qual_years = list(race_quarter_value(race_quarter))
qual_recaps = {}
qual_session = range(1, 4)
zero_time = timedelta(0)
threshold_multiplier = 1.5

for race_number in qual_years:
    for session in qual_session:
        keyword = f'{race_number}_{race_session}_drivers_info_Q{session}'
        driver_data_file = None
        for fname in os.listdir(parent_file + 'data/processed/'):
            if keyword in fname:
                driver_data_file = fname
                break

        if driver_data_file:
            os.chdir(parent_file + 'data/processed/')
            try:
                arr = pd.read_csv(driver_data_file)

                num_rows = len(arr)

                for i in range(num_rows):
                    try:
                        name_driver_1 = arr['Name'][i]
                        name_driver_2 = arr['Name_1'][i]

                        laptime_driver_1_str = arr['LapTime'][i]
                        laptime_driver_2_str = arr['LapTime_1'][i]

                        avg_seconds_driver_1 = parse_laptime_str(laptime_driver_1_str)
                        avg_seconds_driver_2 = parse_laptime_str(laptime_driver_2_str)

                        if (avg_seconds_driver_1 != zero_time and avg_seconds_driver_2 != zero_time
                                and avg_seconds_driver_1 <= threshold_multiplier * avg_seconds_driver_2 and avg_seconds_driver_2 <= threshold_multiplier * avg_seconds_driver_1):

                            time_driver_1 = datetime.strptime(laptime_driver_1_str, '%M:%S.%f')
                            time_driver_2 = datetime.strptime(laptime_driver_2_str, '%M:%S.%f')

                            gap = round((pd.to_timedelta(time_driver_1 - time_driver_2).total_seconds()), 3)

                            corner_adv_1_num, corner_adv_1_den = map(int, arr['corner_advantage'][i].split('/'))
                            corner_advantage_driver_1 = (corner_adv_1_num / corner_adv_1_den) * 100

                            corner_adv_2_num, corner_adv_2_den = map(int, arr['corner_advantage1'][i].split('/'))
                            corner_advantage_driver_2 = (corner_adv_2_num / corner_adv_2_den) * 100

                            qual_recaps.setdefault(name_driver_1, {'total_gap': 0, 'total_corner_advantage': 0, 'count': 0})
                            qual_recaps[name_driver_1]['total_gap'] += gap
                            qual_recaps[name_driver_1]['total_corner_advantage'] += corner_advantage_driver_1
                            qual_recaps[name_driver_1]['count'] += 1

                            qual_recaps.setdefault(name_driver_2, {'total_gap': 0, 'total_corner_advantage': 0, 'count': 0})
                            qual_recaps[name_driver_2]['total_gap'] -= gap  # Invert gap for driver 2
                            qual_recaps[name_driver_2]['total_corner_advantage'] += corner_advantage_driver_2
                            qual_recaps[name_driver_2]['count'] += 1

                    except (KeyError, ValueError) as e:
                        print(f"Error processing row {i} in {driver_data_file}: {e}")
                        continue
                
            except FileNotFoundError:
                print(f"Error: File not found - {driver_data_file}")
            except pd.errors.EmptyDataError:
                print(f"Error: Empty data in file - {driver_data_file}")
            except Exception as e:
                print(f"An error occurred while processing {driver_data_file}: {e}")
            
final_qual_recaps = {}
for name, data in qual_recaps.items():
    count = data['count']
    avg_gap = round(data['total_gap'] / count, 3) if count > 0 else "N/A"
    avg_corner_advantage = round(data['total_corner_advantage'] / count) if count > 0 else "N/A"
    final_qual_recaps[name] = {
        'Average Gap Quali Lap': f'{avg_gap}s',
        'Corner Advantage': f'{avg_corner_advantage}%'
    }


In [3]:
print(final_qual_recaps)

{'Max Verstappen': {'Average Gap Quali Lap': '-0.553s', 'Corner Advantage': '58%'}, 'Liam Lawson': {'Average Gap Quali Lap': '0.343s', 'Corner Advantage': '46%'}, 'Pierre Gasly': {'Average Gap Quali Lap': '-0.264s', 'Corner Advantage': '52%'}, 'Jack Doohan': {'Average Gap Quali Lap': '0.264s', 'Corner Advantage': '48%'}, 'Kimi Antonelli': {'Average Gap Quali Lap': '0.179s', 'Corner Advantage': '55%'}, 'George Russell': {'Average Gap Quali Lap': '-0.179s', 'Corner Advantage': '45%'}, 'Fernando Alonso': {'Average Gap Quali Lap': '-0.288s', 'Corner Advantage': '53%'}, 'Lance Stroll': {'Average Gap Quali Lap': '0.288s', 'Corner Advantage': '47%'}, 'Charles Leclerc': {'Average Gap Quali Lap': '-0.127s', 'Corner Advantage': '56%'}, 'Lewis Hamilton': {'Average Gap Quali Lap': '0.127s', 'Corner Advantage': '44%'}, 'Yuki Tsunoda': {'Average Gap Quali Lap': '0.357s', 'Corner Advantage': '47%'}, 'Isack Hadjar': {'Average Gap Quali Lap': '-0.134s', 'Corner Advantage': '48%'}, 'Alexander Albon': {'